## 1. Imports and loading data

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA as sklearn_pca

np.set_printoptions(formatter={'float':'{:0.4f}'.format})
pd.set_option('display.precision', 5)

filePath = "F:/NortheasternUniversity/Semester4/DataMining/Project/dataset_diabetes/diabetic_data.csv"
# Read dataset using pandas
data = pd.read_csv(filePath, sep=',',header=0)

## 2. Preprocessing
This data set contains both numeric and nominal data types. For numeric data we need to do normalization, while for categorical data, we perform one-hot encoding.

In [2]:
# Drop encounter_id and patient_nbr. Drop weight, payer_code and medical_specialty which has 97% missing data
df = data.drop(['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty',
                'discharge_disposition_id',
                'admission_source_id',
#               'time_in_hospital', 
                'num_lab_procedures', 
#                'num_procedures', 'num_medications', 
                'number_outpatient', 'number_emergency', 'number_inpatient', 
#                'number_diagnoses',
                'max_glu_serum',
                'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 
                'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 
                'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 
                'examide', 'citoglipton',
                'insulin','glyburide-metformin', 'glipizide-metformin', 
                'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change'], axis = 1)
df.head()

,race,gender,age,admission_type_id,time_in_hospital,num_procedures,num_medications,diag_1,diag_2,diag_3,number_diagnoses,A1Cresult,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,1,0,1,250.83,?,?,1,None,No,NO
1,Caucasian,Female,[10-20),1,3,0,18,276,250.01,255,9,None,Yes,>30
2,AfricanAmerican,Female,[20-30),1,2,5,13,648,250,V27,6,None,Yes,NO
3,Caucasian,Male,[30-40),1,2,1,16,8,250.43,403,7,None,Yes,NO
4,Caucasian,Male,[40-50),1,1,0,8,197,157,250,5,None,Yes,NO


In [3]:
# Changed the 'diag_1' based on ICD9 values to one of 'Diabetes','Circulatory','Respiratory','Digestive','Injury','Musculoskeletal','Genitourinary','Neoplasms'
df['diag_1'] = np.where(df['diag_1'].str.contains('250'), 'Diabetes', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['390','391','392','393','394','395','396','397','398','399','400','401','402','403','404','405','406','407','408','409','410','411','412','413','414','415','416','417','418','419','420','421','422','423','424','425','426','427','428','429','430','431','432','433','434','435','436','437','438','439','440','441','442','443','444','445','446','447','448','449','450','451','452','453','454','455','456','457','458','459','785']), 'Circulatory', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['460','461','462','463','464','465','466','467','468','469','470','471','472','473','474','475','476','477','478','479','480','481','482','483','484','485','486','487','488','489','490','49','492','493','494','495','496','497','498','499','500','501','502','503','504','505','506','507','508','509','510','511','512','513','514','515','516','517','518','519','786']), 'Respiratory', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['520','521','522','523','524','525','526','527','528','529','530','531','532','533','534','535','536','537','538','539','540','541','542','543','544','545','546','547','548','549','550','55','552','553','554','555','556','557','558','559','560','561','562','563','564','565','566','567','568','569','570','571','572','573','574','575','576','577','578','579','787']), 'Digestive', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['800','801','802','803','804','805','806','807','808','809','810','811','812','813','814','815','816','817','818','819','820','821','822','823','824','825','826','827','828','829','830','81','832','833','834','835','836','837','838','839','840','841','842','843','844','845','846','847','848','849','850','851','852','853','854','855','856','857','858','859','860','861','862','863','864','865','866','867','868','869','870','871','872','873','874','875','876','877','878','879','880','881','882','883','884','885','886','887','888','889','890','891','892','893','894','895','896','897','898','899','900','901','902','903','904','905','906','907','908','909','910','911','912','913','914','915','916','917','918','919','920','921','922','923','924','925','26','927','928','929','930','931','932','933','934','935','936','937','938','939','940','941','942','943','944','945','946','947','948','949','950','951','952','953','954','955','956','957','958','959','960','961','962','963','964','965','966','967','968','969','970','971','972','973','974','975','976','977','978','979','980','981','982','983','984','985','986','987','988','989','990','991','992','993','994','995','996','997','998','999']), 'Injury', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['710','711','712','713','714','715','716','717','718','719','720','721','722','723','724','725','726','727','728','729','730','731','732','733','734','735','736','737','738','739']), 'Musculoskeletal', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['580','581','582','583','584','585','586','587','588','589','590','591','592','593','594','595','596','597','598','599','600','601','602','603','604','605','606','607','608','609','610','61','612','613','614','615','616','617','618','619','620','621','622','623','624','625','626','627','628','629','788']), 'Genitourinary', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156','157','158','159','160','161','162','163','164','165','166','167','168','169','170','17','172','173','174','175','176','177','178','179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194','195','196','197','198','199','200','201','202','203','204','205','206','207','208','209','210','211','212','213','214','215','216','217','218','219','220','221','222','223','224','225','226','227','228','229','230','231','232','233','234','235','236','237','238']), 'Neoplasms', df['diag_1'])
df['diag_1'] = np.where(df['diag_1'].isin(['Diabetes','Circulatory','Respiratory','Digestive','Injury','Musculoskeletal','Genitourinary','Neoplasms']), df['diag_1'], 'Other')


In [4]:
# Changed the 'diag_2' based on ICD9 values to one of 'Diabetes','Circulatory','Respiratory','Digestive','Injury','Musculoskeletal','Genitourinary','Neoplasms'
df['diag_2'] = np.where(df['diag_2'].str.contains('250'), 'Diabetes', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['390','391','392','393','394','395','396','397','398','399','400','401','402','403','404','405','406','407','408','409','410','411','412','413','414','415','416','417','418','419','420','421','422','423','424','425','426','427','428','429','430','431','432','433','434','435','436','437','438','439','440','441','442','443','444','445','446','447','448','449','450','451','452','453','454','455','456','457','458','459','785']), 'Circulatory', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['460','461','462','463','464','465','466','467','468','469','470','471','472','473','474','475','476','477','478','479','480','481','482','483','484','485','486','487','488','489','490','49','492','493','494','495','496','497','498','499','500','501','502','503','504','505','506','507','508','509','510','511','512','513','514','515','516','517','518','519','786']), 'Respiratory', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['520','521','522','523','524','525','526','527','528','529','530','531','532','533','534','535','536','537','538','539','540','541','542','543','544','545','546','547','548','549','550','55','552','553','554','555','556','557','558','559','560','561','562','563','564','565','566','567','568','569','570','571','572','573','574','575','576','577','578','579','787']), 'Digestive', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['800','801','802','803','804','805','806','807','808','809','810','811','812','813','814','815','816','817','818','819','820','821','822','823','824','825','826','827','828','829','830','81','832','833','834','835','836','837','838','839','840','841','842','843','844','845','846','847','848','849','850','851','852','853','854','855','856','857','858','859','860','861','862','863','864','865','866','867','868','869','870','871','872','873','874','875','876','877','878','879','880','881','882','883','884','885','886','887','888','889','890','891','892','893','894','895','896','897','898','899','900','901','902','903','904','905','906','907','908','909','910','911','912','913','914','915','916','917','918','919','920','921','922','923','924','925','26','927','928','929','930','931','932','933','934','935','936','937','938','939','940','941','942','943','944','945','946','947','948','949','950','951','952','953','954','955','956','957','958','959','960','961','962','963','964','965','966','967','968','969','970','971','972','973','974','975','976','977','978','979','980','981','982','983','984','985','986','987','988','989','990','991','992','993','994','995','996','997','998','999']), 'Injury', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['710','711','712','713','714','715','716','717','718','719','720','721','722','723','724','725','726','727','728','729','730','731','732','733','734','735','736','737','738','739']), 'Musculoskeletal', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['580','581','582','583','584','585','586','587','588','589','590','591','592','593','594','595','596','597','598','599','600','601','602','603','604','605','606','607','608','609','610','61','612','613','614','615','616','617','618','619','620','621','622','623','624','625','626','627','628','629','788']), 'Genitourinary', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156','157','158','159','160','161','162','163','164','165','166','167','168','169','170','17','172','173','174','175','176','177','178','179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194','195','196','197','198','199','200','201','202','203','204','205','206','207','208','209','210','211','212','213','214','215','216','217','218','219','220','221','222','223','224','225','226','227','228','229','230','231','232','233','234','235','236','237','238']), 'Neoplasms', df['diag_2'])
df['diag_2'] = np.where(df['diag_2'].isin(['Diabetes','Circulatory','Respiratory','Digestive','Injury','Musculoskeletal','Genitourinary','Neoplasms']), df['diag_2'], 'Other')


In [5]:
# Changed the 'diag_3' based on ICD9 values to one of 'Diabetes','Circulatory','Respiratory','Digestive','Injury','Musculoskeletal','Genitourinary','Neoplasms'
df['diag_3'] = np.where(df['diag_3'].str.contains('250'), 'Diabetes', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['390','391','392','393','394','395','396','397','398','399','400','401','402','403','404','405','406','407','408','409','410','411','412','413','414','415','416','417','418','419','420','421','422','423','424','425','426','427','428','429','430','431','432','433','434','435','436','437','438','439','440','441','442','443','444','445','446','447','448','449','450','451','452','453','454','455','456','457','458','459','785']), 'Circulatory', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['460','461','462','463','464','465','466','467','468','469','470','471','472','473','474','475','476','477','478','479','480','481','482','483','484','485','486','487','488','489','490','49','492','493','494','495','496','497','498','499','500','501','502','503','504','505','506','507','508','509','510','511','512','513','514','515','516','517','518','519','786']), 'Respiratory', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['520','521','522','523','524','525','526','527','528','529','530','531','532','533','534','535','536','537','538','539','540','541','542','543','544','545','546','547','548','549','550','55','552','553','554','555','556','557','558','559','560','561','562','563','564','565','566','567','568','569','570','571','572','573','574','575','576','577','578','579','787']), 'Digestive', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['800','801','802','803','804','805','806','807','808','809','810','811','812','813','814','815','816','817','818','819','820','821','822','823','824','825','826','827','828','829','830','81','832','833','834','835','836','837','838','839','840','841','842','843','844','845','846','847','848','849','850','851','852','853','854','855','856','857','858','859','860','861','862','863','864','865','866','867','868','869','870','871','872','873','874','875','876','877','878','879','880','881','882','883','884','885','886','887','888','889','890','891','892','893','894','895','896','897','898','899','900','901','902','903','904','905','906','907','908','909','910','911','912','913','914','915','916','917','918','919','920','921','922','923','924','925','26','927','928','929','930','931','932','933','934','935','936','937','938','939','940','941','942','943','944','945','946','947','948','949','950','951','952','953','954','955','956','957','958','959','960','961','962','963','964','965','966','967','968','969','970','971','972','973','974','975','976','977','978','979','980','981','982','983','984','985','986','987','988','989','990','991','992','993','994','995','996','997','998','999']), 'Injury', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['710','711','712','713','714','715','716','717','718','719','720','721','722','723','724','725','726','727','728','729','730','731','732','733','734','735','736','737','738','739']), 'Musculoskeletal', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['580','581','582','583','584','585','586','587','588','589','590','591','592','593','594','595','596','597','598','599','600','601','602','603','604','605','606','607','608','609','610','61','612','613','614','615','616','617','618','619','620','621','622','623','624','625','626','627','628','629','788']), 'Genitourinary', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156','157','158','159','160','161','162','163','164','165','166','167','168','169','170','17','172','173','174','175','176','177','178','179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194','195','196','197','198','199','200','201','202','203','204','205','206','207','208','209','210','211','212','213','214','215','216','217','218','219','220','221','222','223','224','225','226','227','228','229','230','231','232','233','234','235','236','237','238']), 'Neoplasms', df['diag_3'])
df['diag_3'] = np.where(df['diag_3'].isin(['Diabetes','Circulatory','Respiratory','Digestive','Injury','Musculoskeletal','Genitourinary','Neoplasms']), df['diag_3'], 'Other')
df.head()

,race,gender,age,admission_type_id,time_in_hospital,num_procedures,num_medications,diag_1,diag_2,diag_3,number_diagnoses,A1Cresult,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,1,0,1,Diabetes,Other,Other,1,None,No,NO
1,Caucasian,Female,[10-20),1,3,0,18,Other,Diabetes,Other,9,None,Yes,>30
2,AfricanAmerican,Female,[20-30),1,2,5,13,Other,Diabetes,Other,6,None,Yes,NO
3,Caucasian,Male,[30-40),1,2,1,16,Other,Diabetes,Circulatory,7,None,Yes,NO
4,Caucasian,Male,[40-50),1,1,0,8,Neoplasms,Neoplasms,Diabetes,5,None,Yes,NO


In [6]:
# TODO: Currently performed binary encoding. Should try data binning.
# df['diag_1'] = np.where(df['diag_1'].str.contains('250'), 1, 0)
# df['diag_2'] = np.where(df['diag_2'].str.contains('250'), 1, 0)
# df['diag_3'] = np.where(df['diag_3'].str.contains('250'), 1, 0)

In [7]:
# Standardizing numeric data
from sklearn.preprocessing import StandardScaler
numData = df[['time_in_hospital', 
#              'num_lab_procedures', 
              'num_procedures', 'num_medications', 
#              'number_outpatient', 'number_emergency', 'number_inpatient', 
             'number_diagnoses'
             ]]
numData_std = StandardScaler().fit_transform(numData)

In [8]:
# Confirm that the numeric data are standardized
df_numData = pd.DataFrame(numData_std)
df_numData.describe()

,0,1,2,3
count,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05
mean,2.35731e-14,4.23327e-15,-2.18372e-14,1.28417e-13
std,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00
min,-1.13765e+00,-7.85398e-01,-1.84827e+00,-3.32160e+00
25%,-8.02651e-01,-7.85398e-01,-7.40920e-01,-7.35733e-01
50%,-1.32655e-01,-1.99162e-01,-1.25726e-01,2.98612e-01
75%,5.37341e-01,3.87074e-01,4.89467e-01,8.15784e-01
max,3.21732e+00,2.73202e+00,7.99483e+00,4.43599e+00


In [9]:
df[['time_in_hospital', 
#    'num_lab_procedures', 
    'num_procedures', 'num_medications', 
#             'number_outpatient', 'number_emergency', 'number_inpatient', 
    'number_diagnoses'
   ]] = df_numData

In [10]:
# Apply one-hot encoding to categorical columns
oneHotData = pd.get_dummies(df, columns=['race', 'gender', 'age', 
                                         'admission_type_id', 
#                                         'discharge_disposition_id',
 #                                        'admission_source_id', 
                                         'diag_1', 
                                         'diag_2', 'diag_3', 'A1Cresult',
#                                         'change', 
                                         'diabetesMed', 'readmitted'],
                            prefix=['race', 'gender', 'age', 'admission_type_id', 
#                                    'discharge_disposition_id', 
 #                                   'admission_source_id',
                                    'diag_1', 'diag_2', 'diag_3', 'A1Cresult',
 #                                   'change', 
                                    'diabetesMed',
                                    'readmitted'])
oneHotData.describe()

,time_in_hospital,num_procedures,num_medications,number_diagnoses,race_?,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,...,diag_3_Respiratory,A1Cresult_>7,A1Cresult_>8,A1Cresult_None,A1Cresult_Norm,diabetesMed_No,diabetesMed_Yes,readmitted_<30,readmitted_>30,readmitted_NO
count,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,...,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000
mean,2.35731e-14,4.23327e-15,-2.18372e-14,1.28417e-13,0.02234,0.18877,0.00630,0.74778,0.02002,0.01480,...,0.06667,0.03746,0.08073,0.83277,0.04903,0.22997,0.77003,0.11160,0.34928,0.53912
std,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,0.14777,0.39132,0.07911,0.43429,0.14006,0.12075,...,0.24946,0.18988,0.27243,0.37318,0.21594,0.42081,0.42081,0.31487,0.47675,0.49847
min,-1.13765e+00,-7.85398e-01,-1.84827e+00,-3.32160e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,-8.02651e-01,-7.85398e-01,-7.40920e-01,-7.35733e-01,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000
50%,-1.32655e-01,-1.99162e-01,-1.25726e-01,2.98612e-01,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000
75%,5.37341e-01,3.87074e-01,4.89467e-01,8.15784e-01,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,1.00000,1.00000
max,3.21732e+00,2.73202e+00,7.99483e+00,4.43599e+00,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


## 3. PCA

In [11]:
pca = sklearn_pca(n_components=10)
pca.fit_transform(oneHotData)

array([[-3.4558, 1.7840, -1.4736, ..., -0.5294, 0.7686, -0.3634],
       [-0.2256, -1.2500, 0.1122, ..., 0.2310, 0.2399, -0.5329],
       [-0.1418, 1.8791, 0.4749, ..., 1.6069, -0.2232, -0.1496],
       ..., 
       [-0.4583, -2.4392, 2.1381, ..., 0.2775, -0.3364, -0.3549],
       [1.8368, -0.6426, -0.8058, ..., -0.5398, -0.6870, 0.1104],
       [-0.0636, -0.1570, 0.9200, ..., -0.0062, 0.4097, -0.4915]])

In [12]:
print(pca.explained_variance_ratio_)

[0.1874 0.1014 0.0796 0.0520 0.0494 0.0420 0.0367 0.0350 0.0319 0.0282]
